In [ ]:
!pip install tensorflow-serving-api

In [ ]:
import tensorflow as tf
import os
from PIL import Image
import numpy as np
import cv2
import time
import glob
import random
import pandas as pd
from PIL import Image
import PIL
import io
import argparse
import sys
from openvino.inference_engine import IECore
import IPython.display
from IPython.display import clear_output

import datetime
import grpc
from tensorflow import make_tensor_proto, make_ndarray
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc, get_model_metadata_pb2

from tensorflow.keras.applications.resnet import preprocess_input

In [ ]:
class Model(object):

    def __init__(self):
        #Read in Labels
        arg_labels="train_data/labels.txt"
        label_file = open(arg_labels, "r")
        self.labels = label_file.read().split('\n')

    def predict(self, imageFile):
        raise NotImplementedError

class RemodeOpenVINOModel(Model):        
    def __init__(self, grpc_address='localhost', grpc_port=9000, model_name='dogcat', model_version=None):
        super(RemodeOpenVINOModel, self).__init__()
        
        #Settings for accessing model server
        self.grpc_address = grpc_address
        self.grpc_port = grpc_port
        self.model_name = model_name
        self.model_version = model_version
        channel = grpc.insecure_channel("{}:{}".format(self.grpc_address, self.grpc_port))
        self.stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)
        
        # Get input shape info from Model Server
        self.input_name, input_shape, self.output_name, output_shape = self.__get_input_name_and_shape__()
        self.input_height = input_shape[2]
        self.input_width = input_shape[3]
        
    def __get_input_name_and_shape__(self):
        metadata_field = "signature_def"
        request = get_model_metadata_pb2.GetModelMetadataRequest()
        request.model_spec.name = self.model_name
        if self.model_version is not None:
            request.model_spec.version.value = self.model_version
        request.metadata_field.append(metadata_field)

        result = self.stub.GetModelMetadata(request, 10.0) # result includes a dictionary with all model outputs
        input_metadata, output_metadata = self.__get_input_and_output_meta_data__(result)
        input_blob = next(iter(input_metadata.keys()))
        output_blob = next(iter(output_metadata.keys()))
        return input_blob, input_metadata[input_blob]['shape'], output_blob, output_metadata[output_blob]['shape']
    
    def __get_input_and_output_meta_data__(self, response):
        signature_def = response.metadata['signature_def']
        signature_map = get_model_metadata_pb2.SignatureDefMap()
        signature_map.ParseFromString(signature_def.value)
        serving_default = signature_map.ListFields()[0][1]['serving_default']
        serving_inputs = serving_default.inputs
        input_blobs_keys = {key: {} for key in serving_inputs.keys()}
        tensor_shape = {key: serving_inputs[key].tensor_shape
                        for key in serving_inputs.keys()}
        for input_blob in input_blobs_keys:
            inputs_shape = [d.size for d in tensor_shape[input_blob].dim]
            tensor_dtype = serving_inputs[input_blob].dtype
            input_blobs_keys[input_blob].update({'shape': inputs_shape})
            input_blobs_keys[input_blob].update({'dtype': tensor_dtype})
        
        serving_outputs = serving_default.outputs
        output_blobs_keys = {key: {} for key in serving_outputs.keys()}
        tensor_shape = {key: serving_outputs[key].tensor_shape
                        for key in serving_outputs.keys()}
        for output_blob in output_blobs_keys:
            outputs_shape = [d.size for d in tensor_shape[output_blob].dim]
            tensor_dtype = serving_outputs[output_blob].dtype
            output_blobs_keys[output_blob].update({'shape': outputs_shape})
            output_blobs_keys[output_blob].update({'dtype': tensor_dtype})

        return input_blobs_keys, output_blobs_keys
    
    def predict(self, imageFile):
        start1 = time.time() #ここ追加
        
        image = cv2.imread(imageFile)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (self.input_width, self.input_height))
        frame = image
        image = preprocess_input(image)
        image = image.transpose((2, 0, 1))  # Change data layout from HWC to CHW
        image = image.reshape((1, 3, self.input_height, self.input_width))
        images = image
        input_image = images.astype(np.float32)

        start2 = time.time() #ここ追加
        #predictions = self.exec_net.infer(inputs={self.input_blob: images})
        # Model ServerにgRPCでアクセスしてモデルをコール
        request = predict_pb2.PredictRequest()
        request.model_spec.name = self.model_name
        request.inputs[self.input_name].CopyFrom(make_tensor_proto(input_image, shape=(input_image.shape)))
        result = self.stub.Predict(request, 10.0) # result includes a dictionary with all model outputs
        predictions = make_ndarray(result.outputs[self.output_name])
        
        infer_time = time.time() - start2

        # Print the highest probability label
        #predictions = predictions[self.out_blob]
        highest_probability_index = predictions[0].argsort()[-1:][::-1][0]

        total_time = time.time() - start1

        #return total_time, infer_time, self.labels[highest_probability_index], frame  #ここ追加
        return total_time, infer_time, "", frame  #ここ追加


def run_inference(grpc_address='161.202.224.155', grpc_port='9000', model_name='dogcat', total=500):
    model = RemodeOpenVINOModel(grpc_address, grpc_port, model_name)

    total_infer_spent_time = 0
    total_spent_time = 0
    list_df = pd.DataFrame( columns=['正解ラベル','予測ラベル','全処理時間(msec)','推論時間(msec)'] )

    match = 0
    #file_list = glob.glob(os.path.join(dataset_dir, "*"))
    file_list = glob.glob("train_data/test/*/*")
    for i in range(total):
        img_path = random.choice(file_list)
        img_cat = os.path.split(os.path.dirname(img_path))[1]
        total_time, infer_time, pred_label, frame = model.predict(img_path)

        if i > 1:
            total_infer_spent_time += infer_time
            total_spent_time += total_time

        #print(img_path, str(int(total_time*1000.0)) + 'msec', str(int(infer_time*1000.0)) + 'msec', pred_label) #ここ追加
        clear_output(wait=True)
        frame = cv2.imread(img_path)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        if frame.shape[:-1] != (224, 224):
            frame = cv2.resize(frame, (224, 224))
        cv2.putText(frame,'No ' + str(i+1) + ':' + str(int(total_time*1000)) + ',' + str(int(infer_time*1000)), (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,2550), 4)
        cv2.putText(frame,'No ' + str(i+1) + ':' + str(int(total_time*1000)) + ',' + str(int(infer_time*1000)), (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2)
        cv2.putText(frame,str(img_cat), (10,80), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,2550), 4)
        cv2.putText(frame,str(img_cat), (10,80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2)
        cv2.putText(frame,str(pred_label), (10,130), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,2550), 4)
        cv2.putText(frame,str(pred_label), (10,130), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2)
        f = io.BytesIO()
        PIL.Image.fromarray(frame).save(f, 'jpeg')
        IPython.display.display(IPython.display.Image(data=f.getvalue()))

        tmp_se = pd.Series( [img_cat, pred_label, str(int(total_time * 1000)), str(int(infer_time * 1000)) ], index=list_df.columns )
        list_df = list_df.append( tmp_se, ignore_index=True ) 

    print()
    print('全' + str(total) + '枚 完了！')
    print()
    print("平均処理時間: " + str(int((total_spent_time / (total-1))*1000.0)) + " ms/枚")
    print("平均推論時間: " + str(int((total_infer_spent_time / (total-1))*1000.0)) + " ms/枚")
    return int((total_spent_time / (total-1))*1000.0), int((total_infer_spent_time / (total-1))*1000.0)

In [ ]:
run_inference('161.202.224.155', '9000', 'dogcat', 100)